In [1]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [2]:
from aiida.manage.configuration import get_profile

from aiida.common.datastructures import CalcInfo, CodeInfo
from aiida.common.folders import Folder

from aiida.engine import CalcJob, WorkChain
from aiida.engine import calcfunction, workfunction, submit, run

from aiida.plugins import DataFactory, WorkflowFactory

from aiida.parsers.parser import Parser
from aiida.orm import Code
from aiida.orm import load_code, load_node
from aiida.orm import Str, Dict, Float, Int
from aiida.orm import QueryBuilder

import aiida

from alamode_aiida.data_loader import load_anphon_kl, load_anphon_kl_spec
from ase.io.espresso import write_espresso_in
from ase.io.lammpsdata import read_lammps_data
from ase.build import make_supercell
from ase import Atom, Atoms
import ase

from itertools import combinations_with_replacement
import numpy as np
import os
import subprocess
import shutil
import re
from time import sleep
import spglib
import json

from os.path import expanduser

from tools import NodeBank
from tools import wait_for_node_finished
from alamode_aiida.ase_support import load_atoms
from tools.putstructure import PutStructure


In [3]:
# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')


In [4]:
g_graph = False

with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


{'CWD': 'run27'}


'/home/max/Documents/aiida-test/alamode-aiida/example/run27'

In [5]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os
home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
code = get_or_create_code('alamode.alm_suggest',
                          computer_local,
                          '/home/max/Documents/alamode/bin/alm')
code_alm_sugget = code

code_alm_sugget


<Code: Remote code 'alamode.alm_suggest' on localhost, pk: 41706, uuid: 06f551d9-68cd-40d2-8e61-bc0a380843ed>

In [6]:
# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [7]:
# 計算するroot directoryの定義
key = 'root_wd'
g_root_wd = nodebank.load(key)

In [8]:
# harmonic partの計算を行う。
g_norder = nodebank.load_or_dump("norder", Int(1))
g_norder

debug, node uuid: a4bf90fb-b26a-452f-a207-0431a7b553fd (unstored) value: 1
debug, load None
debug, force dump


<Int: uuid: a4bf90fb-b26a-452f-a207-0431a7b553fd (pk: 51412) value: 1>

In [9]:
# g_primstructure_filepath = nodebank.load("primstructure_filepath")
# g_superstructure_filepath = nodebank.load("superstructure_filepath")
# g_superstructure_file = nodebank.load("superstructure_file")
g_primstructure = nodebank.load("primstructure")
g_superstructure = nodebank.load("superstructure")

g_primstructure_filename = nodebank.load("primstructure_filename")
g_superstructure_filename = nodebank.load("superstructure_filename")

In [10]:
g_superstructure.get_ase()

Atoms(symbols='Ga128N128', pbc=True, cell=[[12.76, 0.0, 0.0], [-6.38, 11.050484152289, 0.0], [0.0, 0.0, 20.756]], masses=...)

In [11]:
g_cwd = nodebank.load("cwd")
g_cwd

<Str: uuid: facb1820-88fc-426f-a8e0-681ec484ca38 (pk: 51410) value: /home/max/Documents/aiida-test/alamode-aiida/example/run27/Ga2N2_1x1x1_harmonic>

In [12]:
g_almprefix= nodebank.load("almprefix")

## alm suggest

In [13]:
# 一意のファイル名にするためのprefix文字列
# norder=1なのでharmonicとする
g_prefix = nodebank.load_or_dump("prefix", Str("harmonic"))


debug, node uuid: 2ce72424-aa0e-4766-a47a-82b7eac76291 (unstored) value: harmonic
debug, load None
debug, force dump


# ALM

In [14]:
from alm import ALM

In [15]:
@calcfunction
def alm_sugget(superstructure, norder):
    lavec = superstructure.get_ase().cell
    xcoord = superstructure.get_ase().get_scaled_positions()
    kd = superstructure.get_ase().get_atomic_numbers()
    displacement_patterns = []
    with ALM(lavec, xcoord, kd) as alm:
        alm.define(norder.value)
        alm.suggest()
        print(alm.getmap_primitive_to_supercell())
        
        for fc_order in range(1, 1+g_norder.value):
            print(fc_order)
            print(alm.get_displacement_patterns(fc_order))
            displacement_patterns.append(alm.get_displacement_patterns(fc_order))
            
    return List(list=displacement_patterns)

@calcfunction
def alm_sugget_put_pattern_files(displacement_patterns, cwd, filename_list):
    folderdata = FolderData()
    for displacement_pattern, filename in zip(displacement_patterns, filename_list):
        filepath = os.path.abspath(os.path.join(cwd.value, filename))
        with open(filepath,"w") as f:
            json.dump(displacement_pattern, f)
        folderdata.put_object_from_file(filepath, filename)
    return folderdata

displacement_patterns = alm_sugget( g_superstructure, g_norder)

filename_list = List(list=[Str("pattern_dict.HARMONIC")])
adisplacement_patterns_folder_data = alm_sugget_put_pattern_files(displacement_patterns, 
                                                                  g_cwd, 
                                   filename_list)

[[  0   1   2   3]
 [  4   5   6   7]
 [  8   9  10  11]
 [ 12  13  14  15]
 [ 16  17  18  19]
 [ 20  21  22  23]
 [ 24  25  26  27]
 [ 28  29  30  31]
 [ 32  33  34  35]
 [ 36  37  38  39]
 [ 40  41  42  43]
 [ 44  45  46  47]
 [ 48  49  50  51]
 [ 52  53  54  55]
 [ 56  57  58  59]
 [ 60  61  62  63]
 [ 64  65  66  67]
 [ 68  69  70  71]
 [ 72  73  74  75]
 [ 76  77  78  79]
 [ 80  81  82  83]
 [ 84  85  86  87]
 [ 88  89  90  91]
 [ 92  93  94  95]
 [ 96  97  98  99]
 [100 101 102 103]
 [104 105 106 107]
 [108 109 110 111]
 [112 113 114 115]
 [116 117 118 119]
 [120 121 122 123]
 [124 125 126 127]
 [128 129 130 131]
 [132 133 134 135]
 [136 137 138 139]
 [140 141 142 143]
 [144 145 146 147]
 [148 149 150 151]
 [152 153 154 155]
 [156 157 158 159]
 [160 161 162 163]
 [164 165 166 167]
 [168 169 170 171]
 [172 173 174 175]
 [176 177 178 179]
 [180 181 182 183]
 [184 185 186 187]
 [188 189 190 191]
 [192 193 194 195]
 [196 197 198 199]
 [200 201 202 203]
 [204 205 206 207]
 [208 209 21

In [16]:
nodebank.dump('displacement_patterns', displacement_patterns)

<List: uuid: 05880804-5925-44c1-88b8-926b4c7de74c (pk: 51415) value: [[[[0, [1.0, 0.0, 0.0], 'Cartesian']], [[0, [0.0, 1.0, 0.0], 'Cartesian']], [[0, [0.0, -1.0, 0.0], 'Cartesian']], [[0, [0.0, 0.0, 1.0], 'Cartesian']], [[0, [0.0, 0.0, -1.0], 'Cartesian']], [[2, [1.0, 0.0, 0.0], 'Cartesian']], [[2, [0.0, 1.0, 0.0], 'Cartesian']], [[2, [0.0, -1.0, 0.0], 'Cartesian']], [[2, [0.0, 0.0, 1.0], 'Cartesian']], [[2, [0.0, 0.0, -1.0], 'Cartesian']]]]>

In [17]:
@calcfunction
def make_pattern_files(displacement_patterns, cwd, filename_list):
    folderdata = FolderData()
    cwd = cwd.value
    for displacement_pattern, filename in zip(displacement_patterns.get_list(), filename_list.get_list()):
        filename = filename
        print(filename)
        basis = []
        for pats in displacement_pattern:
            for pat in pats:
                basis.append(pat[-1])
        basis = np.array(basis)
        if np.all(basis=='Cartesian'):
            basis = "C"
        else:
            raise ValueError('not all basis is C')
        lines = [f"basis : {basis}"]
        for i, pats in enumerate(displacement_pattern):
            lines.append(f'{i+1}: {len(pats)}')
            for pat in pats:
                n = pat[0]
                v = list(map(int,pat[1]))
                lines.append(f' {n+1} {v[0]} {v[1]} {v[2]}')
        # write to the file
        filepath = os.path.join(cwd, filename) 
        with open(filepath, "w") as f:
            f.write("\n".join(lines))
        print(filepath,"is made.")
        folderdata.put_object_from_file(filepath, filename)
    return folderdata
        
filename_list = List(list=['pattern.harmonic','pattern.cubic'])
pattern_folder = make_pattern_files(displacement_patterns, g_cwd, filename_list)

pattern.harmonic
/home/max/Documents/aiida-test/alamode-aiida/example/run27/Ga2N2_1x1x1_harmonic/pattern.harmonic is made.


In [18]:
pattern_folder.list_object_names()

['pattern.harmonic']